In [36]:
# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import scipy.stats as stats

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
%matplotlib inline

In [37]:
# Read in county information
counties = pd.read_csv('data/acs2017_county_data.csv')

In [38]:
# Look at it!
counties.columns

Index(['CountyId', 'State', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'VotingAgeCitizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment'],
      dtype='object')

In [39]:
# Check to see if '(city)' appears in any rows
for index, row in counties.iterrows():
    if '(city)' in row.County.split():
        print(row.County)

In [40]:
states_abrv = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
              "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
              "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
              "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
              "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado", 
              "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois", 
              "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland", 
              "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana", 
              "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
              "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania", 
              "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah", 
              "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]
states_dict = dict(zip(states, states_abrv))

In [41]:
counties['State_abrv'] = counties['State'].map(states_dict)

In [42]:
# See if 'city' appears in any rows
for index, row in counties.iterrows():
    if 'city' in row.County.split():
        print(row.County)

Baltimore city
St. Louis city
Alexandria city
Bristol city
Buena Vista city
Charlottesville city
Chesapeake city
Colonial Heights city
Covington city
Danville city
Emporia city
Fairfax city
Falls Church city
Franklin city
Fredericksburg city
Galax city
Hampton city
Harrisonburg city
Hopewell city
Lexington city
Lynchburg city
Manassas city
Manassas Park city
Martinsville city
Newport News city
Norfolk city
Norton city
Petersburg city
Poquoson city
Portsmouth city
Radford city
Richmond city
Roanoke city
Salem city
Staunton city
Suffolk city
Virginia Beach city
Waynesboro city
Williamsburg city
Winchester city


In [43]:
# It does, so we need to capitalize it
counties['County'] = counties.County.apply(lambda x: ' '.join(x.split()[:-1]) + ' City' \
                                            if x.split()[-1] == 'city' else x)

In [44]:
# Notice how 'city' doesn't appear in any rows anymore
for index, row in counties.iterrows():
    if 'city' in row.County.split():
        print(row.County)

In [45]:
# Now, let's add 'County' to the end of these county names:
counties['County'] = counties.County.apply(lambda x: ' '.join(x.split()) + ', County' \
                                            if x.split()[-1] != 'County' else x)

In [46]:
counties['County'] = counties['County'] + ', ' + counties['State_abrv']

In [47]:
counties.sample(20)

,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,State_abrv
1748,32003,Nevada,"Clark County, NV",2112436,1056002,1056434,30.7,44.1,10.8,0.4,...,2.3,3.6,24.5,982033,84.5,10.5,4.9,0.1,8.3,NV
438,13105,Georgia,"Elbert County, GA",19288,9218,10070,5.5,63.5,30.5,0.0,...,0.1,1.8,20.8,7327,72.1,19.3,8.4,0.2,7.7,GA
2559,48073,Texas,"Cherokee County, TX",51594,26307,25287,22.5,61.3,13.6,0.5,...,0.8,2.7,24.4,20526,73.1,20.0,6.8,0.2,6.1,TX
1453,28105,Mississippi,"Oktibbeha County, MS",49392,24860,24532,1.6,56.9,37.1,0.1,...,0.8,3.8,19.9,20717,58.6,37.2,4.0,0.2,9.5,MS
246,8005,Colorado,"Arapahoe County, CO",626612,309190,317422,18.9,61.3,10.2,0.3,...,1.3,6.8,27.7,330548,83.0,11.3,5.5,0.2,5.2,CO
585,16069,Idaho,"Nez Perce County, ID",40014,19811,20203,3.7,87.2,0.4,5.4,...,1.6,4.9,16.0,18977,75.4,17.1,7.3,0.2,4.2,ID
77,2110,Alaska,"Juneau City and Borough, County, AK",32434,16663,15771,6.4,65.4,1.0,10.8,...,2.3,4.3,15.6,17970,53.4,38.3,8.2,0.1,4.8,AK
1044,21103,Kentucky,"Henry County, KY",15677,7715,7962,3.2,91.8,3.5,0.3,...,0.6,3.5,30.5,7143,79.4,12.1,8.2,0.3,6.5,KY
1477,28153,Mississippi,"Wayne County, MS",20458,10011,10447,0.6,57.8,40.5,0.3,...,3.9,7.1,33.6,7800,74.7,15.2,10.1,0.0,10.4,MS
1759,32027,Nevada,"Pershing County, NV",6661,4337,2324,23.8,66.1,4.7,2.8,...,0.7,4.8,16.7,2211,64.9,28.9,6.2,0.0,4.2,NV


In [48]:
# Ok, dank! Now let's read in shooting data.
shootings = pd.read_pickle('data/shootings_by_county_state.pkl')
shootings.head()

,County,num_shootings
0,"Los Angeles County, CA",106
1,"Maricopa County, AZ",65
2,"Cook County, IL",33
3,"Harris County, TX",32
4,"San Bernardino County, CA",30


In [49]:
# Strip whitespace from the two columns in each dataframe that we are merging upon
shootings['County'] = shootings['County'].str.strip()
counties['County'] = counties['County'].str.strip()

In [50]:
# Let's join the shootings onto the counties dataframe on the 'County' column
counties2 = pd.merge(counties, shootings, how = 'left', on='County')

In [51]:
# Let's look at it!
counties2.sort_values('num_shootings', ascending = False)

,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,State_abrv,num_shootings
204,6037,California,"Los Angeles County, CA",10105722,4979641,5126081,48.4,26.5,7.9,0.2,...,5.3,30.9,4805817,79.3,11.2,9.3,0.2,7.8,CA,106.0
103,4013,Arizona,"Maricopa County, AZ",4155501,2055464,2100037,30.6,56.3,5.1,1.5,...,6.4,25.8,1929015,82.8,11.1,5.9,0.1,6.0,AZ,65.0
610,17031,Illinois,"Cook County, IL",5238541,2540704,2697837,25.1,42.7,23.4,0.1,...,4.5,32.9,2521437,84.2,11.1,4.6,0.1,8.7,IL,33.0
2623,48201,Texas,"Harris County, TX",4525519,2251060,2274459,42.2,30.6,18.5,0.2,...,3.7,28.9,2180392,83.4,9.8,6.6,0.2,6.4,TX,32.0
221,6071,California,"San Bernardino County, CA",2121220,1055170,1066050,52.3,29.8,8.0,0.3,...,4.7,30.9,869658,77.3,16.2,6.4,0.1,9.9,CA,30.0
1748,32003,Nevada,"Clark County, NV",2112436,1056002,1056434,30.7,44.1,10.8,0.4,...,3.6,24.5,982033,84.5,10.5,4.9,0.1,8.3,NV,28.0
218,6065,California,"Riverside County, CA",2355002,1171711,1183291,48.0,36.6,6.0,0.4,...,5.2,33.1,978726,77.7,14.7,7.5,0.2,9.9,CA,27.0
215,6059,California,"Orange County, CA",3155816,1558245,1597571,34.2,41.4,1.6,0.2,...,5.6,27.4,1560997,82.0,10.1,7.7,0.2,5.8,CA,23.0
362,12086,Florida,"Miami-Dade County, FL",2702602,1311997,1390605,67.5,13.7,16.3,0.1,...,4.9,31.3,1272735,82.4,9.6,7.8,0.2,7.4,FL,23.0
222,6073,California,"San Diego County, CA",3283665,1651147,1632518,33.4,46.2,4.7,0.4,...,7.0,25.7,1536073,77.8,14.2,7.8,0.2,7.1,CA,22.0


In [52]:
# How big is this new dataframe?
counties2.shape

(3220, 39)

In [53]:
# Drop counties where the name of the county is NaN (mainly Puerto Rico)
counties2.dropna(axis = 0, subset = ['County'], inplace = True)

In [54]:
# Check for size again
counties2.shape

(3141, 39)

In [55]:
# Fill NaNs in num_shootings column with 0 (because they had 0 shootings)
counties2['num_shootings'] = counties2['num_shootings'].fillna(value=0)

In [56]:
# Let's take a look!
counties2.sort_values('num_shootings', ascending = False)

,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,State_abrv,num_shootings
204,6037,California,"Los Angeles County, CA",10105722,4979641,5126081,48.4,26.5,7.9,0.2,...,5.3,30.9,4805817,79.3,11.2,9.3,0.2,7.8,CA,106.0
103,4013,Arizona,"Maricopa County, AZ",4155501,2055464,2100037,30.6,56.3,5.1,1.5,...,6.4,25.8,1929015,82.8,11.1,5.9,0.1,6.0,AZ,65.0
610,17031,Illinois,"Cook County, IL",5238541,2540704,2697837,25.1,42.7,23.4,0.1,...,4.5,32.9,2521437,84.2,11.1,4.6,0.1,8.7,IL,33.0
2623,48201,Texas,"Harris County, TX",4525519,2251060,2274459,42.2,30.6,18.5,0.2,...,3.7,28.9,2180392,83.4,9.8,6.6,0.2,6.4,TX,32.0
221,6071,California,"San Bernardino County, CA",2121220,1055170,1066050,52.3,29.8,8.0,0.3,...,4.7,30.9,869658,77.3,16.2,6.4,0.1,9.9,CA,30.0
1748,32003,Nevada,"Clark County, NV",2112436,1056002,1056434,30.7,44.1,10.8,0.4,...,3.6,24.5,982033,84.5,10.5,4.9,0.1,8.3,NV,28.0
218,6065,California,"Riverside County, CA",2355002,1171711,1183291,48.0,36.6,6.0,0.4,...,5.2,33.1,978726,77.7,14.7,7.5,0.2,9.9,CA,27.0
215,6059,California,"Orange County, CA",3155816,1558245,1597571,34.2,41.4,1.6,0.2,...,5.6,27.4,1560997,82.0,10.1,7.7,0.2,5.8,CA,23.0
362,12086,Florida,"Miami-Dade County, FL",2702602,1311997,1390605,67.5,13.7,16.3,0.1,...,4.9,31.3,1272735,82.4,9.6,7.8,0.2,7.4,FL,23.0
2579,48113,Texas,"Dallas County, TX",2552213,1257751,1294462,39.6,30.2,22.1,0.2,...,4.3,27.2,1252101,84.3,9.1,6.4,0.2,5.9,TX,22.0


In [58]:
counties2['shootings_per_cap'] = counties2['num_shootings']/counties2['TotalPop']
counties2.mean()

CountyId              30389.820121
TotalPop             101984.086597
Men                   50206.847182
Women                 51777.239414
Hispanic                  9.121012
White                    76.772620
Black                     8.884177
Native                    1.812703
Asian                     1.319516
Pacific                   0.085482
VotingAgeCitizen      72086.308500
Income                49745.283031
IncomeErr              3177.098058
IncomePerCap          26031.744031
IncomePerCapErr        1532.221585
Poverty                  15.987202
ChildPoverty             22.105669
Professional             31.527794
Service                  18.123973
Office                   21.779911
Construction             12.639573
Production               15.930118
Drive                    79.539032
Carpool                   9.900000
Transit                   0.925820
Walk                      3.232283
OtherTransp               1.601847
WorkAtHome                4.802674
MeanCommute         

In [59]:
# Create new column based on whether or not there were shootings in a given county
counties2['shootings_abv_avg'] = np.where(counties2['shootings_per_cap'] >= 0.000007, 1, 0)

In [60]:
counties2.sample(20)

,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,State_abrv,num_shootings,shootings_per_cap,shootings_abv_avg
1075,21165,Kentucky,"Menifee County, KY",6381,3382,2999,0.9,97.8,0.1,0.0,...,2529,70.1,23.5,6.4,0.0,5.4,KY,0.0,0.000000,0
1027,21069,Kentucky,"Fleming County, KY",14515,7155,7360,1.2,96.1,1.7,0.0,...,5824,73.8,17.1,9.1,0.0,9.5,KY,0.0,0.000000,0
2756,48467,Texas,"Van Zandt County, TX",53607,26307,27300,10.3,84.3,2.6,0.5,...,22384,74.5,15.0,10.3,0.3,5.9,TX,0.0,0.000000,0
2944,51760,Virginia,"Richmond City, County, VA",220892,104853,116039,6.5,40.0,47.9,0.2,...,109362,79.6,16.1,4.2,0.1,8.8,VA,0.0,0.000000,0
2164,40067,Oklahoma,"Jefferson County, OK",6270,3158,3112,10.0,78.6,1.3,6.9,...,2280,71.9,16.9,11.1,0.1,7.2,OK,0.0,0.000000,0
1297,26133,Michigan,"Osceola County, MI",23221,11725,11496,1.7,94.9,1.0,0.8,...,8960,81.0,10.2,8.4,0.4,7.2,MI,1.0,0.000043,1
2221,41027,Oregon,"Hood River County, OR",22938,11512,11426,31.1,64.2,0.1,0.6,...,11263,78.3,12.4,9.3,0.0,5.1,OR,0.0,0.000000,0
1111,21237,Kentucky,"Wolfe County, KY",7251,3522,3729,0.3,98.2,0.3,0.4,...,1895,61.5,32.1,4.7,1.7,10.5,KY,0.0,0.000000,0
2717,48389,Texas,"Reeves County, TX",14791,8938,5853,75.0,18.2,4.5,0.4,...,5140,70.3,25.9,3.8,0.0,8.0,TX,0.0,0.000000,0
845,19113,Iowa,"Linn County, IA",220008,108534,111474,3.1,87.0,4.8,0.1,...,117384,85.6,9.6,4.6,0.1,3.8,IA,2.0,0.000009,1


In [61]:
counties2.to_pickle('data/combined_data_cor.pkl')

In [62]:
counties2.columns

Index(['CountyId', 'State', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'VotingAgeCitizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment', 'State_abrv',
       'num_shootings', 'shootings_per_cap', 'shootings_abv_avg'],
      dtype='object')

In [63]:
counties2.columns

Index(['CountyId', 'State', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'VotingAgeCitizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment', 'State_abrv',
       'num_shootings', 'shootings_per_cap', 'shootings_abv_avg'],
      dtype='object')